In [1]:
import os
import pandas as pd
data_dir = "data"
def add_train_use(data_dir, df=None):
    if df is None:
        df = pd.read_csv(f"{data_dir}/metadata.csv")
    train_files = [f for f in os.listdir(f"{data_dir}/train/malade/") if f[-4:]=='.png']
    train_files.extend(
        [f for f in os.listdir(f"{data_dir}/train/sain/") if f[-4:]=='.png']
    )
    valid_files = [f for f in os.listdir(f"{data_dir}/valid/malade/") if f[-4:]==".png"]
    valid_files.extend(
        [f for f in os.listdir(f"{data_dir}/valid/sain/") if f[-4:]==".png"]
    )
    df["train"] = df["Image Index"].apply(lambda x: x in train_files)
    return df

In [2]:
df = add_train_use(data_dir, df=None)

In [3]:
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],WEIGHTS,train
0,00000018_000.png,No Finding,0,18,75,M,PA,2500,2048,0.168000,0.168000,1,True
1,00000056_000.png,Nodule|Pneumonia,0,56,76,M,PA,2500,2048,0.168000,0.168000,1,True
2,00000056_001.png,Effusion,1,56,76,M,PA,2992,2991,0.143000,0.143000,1,True
3,00000104_000.png,Pleural_Thickening,0,104,68,F,PA,2906,2991,0.143000,0.143000,1,True
4,00000104_001.png,No Finding,1,104,69,F,PA,2734,2991,0.143000,0.143000,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5238,00030749_000.png,No Finding,0,30749,22,F,PA,2021,2021,0.194311,0.194311,1,True
5239,00030754_000.png,Infiltration,0,30754,54,M,PA,2020,2021,0.194314,0.194314,1,True
5240,00030792_000.png,No Finding,0,30792,10,F,PA,1775,1712,0.194311,0.194311,1,True
5241,00030792_001.png,No Finding,1,30792,10,F,PA,1663,1675,0.194311,0.194311,1,True


In [4]:
import pandas as pd
import os
import shutil
def select_data(data_dir, df=None):
    if df is None:
        df = pd.read_csv(f"{data_dir}/metadata.csv")
    patient_ids = df["Patient ID"].unique().tolist()
    select_names = []
    # We look at each patient individually
    for patient in patient_ids:
        df_patient = df[df["Patient ID"]==patient]
        labels = df_patient["Finding Labels"].unique().tolist()
        # We look at all the labels the patients got
        # If he got always the same label we select the first row
        if len(labels)==1:
            select_names.append(df_patient["Image Index"].values[0])
        else:
        # Else we select the first row with a finding
            if "No Finding" in labels:
                labels.remove("No Finding")
            select_names.append(df_patient[df_patient["Finding Labels"]==labels[0]]["Image Index"].values[0])
    print(len(select_names))

    # Create the folders for the selected data
    os.mkdir(f"{data_dir}/selected_data/")
    os.mkdir(f"{data_dir}/selected_data/train/")
    os.mkdir(f"{data_dir}/selected_data/train/malade/")
    os.mkdir(f"{data_dir}/selected_data/train/sain/")
    os.mkdir(f"{data_dir}/selected_data/valid/")
    os.mkdir(f"{data_dir}/selected_data/valid/malade/")
    os.mkdir(f"{data_dir}/selected_data/valid/sain/")

    # Copy only the selected files
    train_malade_files = [f for f in os.listdir(f"{data_dir}/train/malade/") if f[-4:]=='.png']
    train_sain_files = [f for f in os.listdir(f"{data_dir}/train/sain/") if f[-4:]=='.png']
    valid_malade_files = [f for f in os.listdir(f"{data_dir}/valid/malade/") if f[-4:]=='.png']
    valid_sain_files = [f for f in os.listdir(f"{data_dir}/valid/sain/") if f[-4:]=='.png']
    for f in train_malade_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/train/malade/{f}", f"{data_dir}/selected_data/train/malade/")
    for f in train_sain_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/train/sain/{f}", f"{data_dir}/selected_data/train/sain/")
    for f in valid_malade_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/valid/malade/{f}", f"{data_dir}/selected_data/valid/malade/")
    for f in valid_sain_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/valid/sain/{f}", f"{data_dir}/selected_data/valid/sain/")

    # Save the sub part of the metadata file
    df[df["Image Index"].isin(select_names)].to_csv(f"{data_dir}/selected_data/metadata.csv", index=None)